In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly
from IPython.display import display, HTML
from plotly.graph_objs import *

plotly.offline.init_notebook_mode(connected=True)

def pendulum_dynamics(state, mass=1.0, length=1.0, damping=0.0, non_equiv_offset=0.0):
    g = 9.81  # gravitational constant (m/s^2)
    
    q, q_dot = state  # unpack state variables
    
    non_equivariant_force = - non_equiv_offset * q if q > 0 else 0
    q_ddot = - (g / (length * mass)) * np.sin(q) - (damping) * q_dot + non_equivariant_force / mass
    
    return np.array([q_dot, q_ddot])

def evolve_dynamics(state0, time=1, dt=0.1, mass=1.0, length=1.0, damping=0.0, non_equiv_offset=0.0):
    trajectory = [state0]
    T = np.arange(0, time, dt)
    for _ in T:
        state_dot = pendulum_dynamics(state=trajectory[-1], mass=mass, length=length, damping=damping, non_equiv_offset=non_equiv_offset)
        state = trajectory[-1] + dt * state_dot
        trajectory.append(state)
    return np.array(trajectory).T

def pendulum_kinetic_energy(state, mass=1.0, length=1.0, damping=0.0):    
    _, q_dot = state  # unpack state variables
    return 0.5 * mass * length**2 * q_dot**2

def pendulum_work_function(state, mass=1.0, length=1.0, damping=0.0, non_equiv_offset=0.0):
    g = 9.81  # gravitational constant (m/s^2)
    q, q_dot = state  # unpack state variables

    height = length - length * np.cos(q)  # Height measured from the lowest point
    gravitational_work = mass * g * height

    # Work done against damping
    damping_work = 0.5 * damping * q_dot ** 2 

    non_equiv_work = - 0.5 * (non_equiv_offset * q**2 * (q > 0))
    return gravitational_work + damping_work + non_equiv_work

def lagrangian(state: np.array, mass=1.0, length=1.0, damping=0.0, non_equiv_offset=0.0):
    K = pendulum_kinetic_energy(state, mass=mass, length=length, damping=damping)
    U = pendulum_work_function(state, mass=mass, length=length, damping=damping, non_equiv_offset=non_equiv_offset)
    return K + U

def generate_trajectory_variation(trajectory, epsilon=0.1, num_sinusoidal=3):
    # Set seed to num_sinusoidal for reproducibility
    np.random.seed(num_sinusoidal)
    # Extract the position q from the state
    q_trajectory = trajectory[0, :]
    
    # Number of points in the trajectory
    n_points = len(q_trajectory)
    
    # Time array (assuming trajectory length is 1 second)
    time_array = np.linspace(0, 1, n_points)
    
    # Generate random amplitudes and harmonic multipliers
    amplitudes = np.random.uniform(0, 1, num_sinusoidal)
    max_amplitude = np.max(amplitudes)
    amplitudes = epsilon * amplitudes / max_amplitude
    harmonic_multipliers = np.random.randint(1, max(n_points // 10, num_sinusoidal), num_sinusoidal)
    frequencies = harmonic_multipliers * 2 * np.pi
    
    # Generate sinusoidal variations and their derivatives
    sinusoidal_variations = amplitudes[:, np.newaxis] * np.sin(frequencies[:, np.newaxis] * time_array)
    variation_derivatives = amplitudes[:, np.newaxis] * frequencies[:, np.newaxis] * np.cos(frequencies[:, np.newaxis] * time_array)
 
    # Sum along the rows to get the total variation
    var_q = np.sum(sinusoidal_variations, axis=0)
    var_dq = np.sum(variation_derivatives, axis=0)
    
    # Assert that the variation vanishes at the endpoints
    assert np.isclose(var_q[0], 0, atol=1e-10) and np.isclose(var_q[-1], 0, atol=1e-10), "Variation does not vanish at endpoints"
    
    return var_q, var_dq

In [2]:
import escnn

G = escnn.group.CyclicGroup(2)
rep_s = G.irrep(1) + G.irrep(1)

In [3]:
state0 = [np.pi / 2, 0.1]
time = 0.5

# Reflection action 
g = G.elements[-1]

state_trajectory = evolve_dynamics(state0=state0, time=time, dt=0.005)
traj_lagrangian = lagrangian(state_trajectory)
var_q, var_dq = generate_trajectory_variation(state_trajectory, epsilon=0.005, num_sinusoidal=2)
var_state_trajectory = state_trajectory + np.array([var_q, var_dq])

state_trajectory = var_state_trajectory
g_state_trajectory = evolve_dynamics(state0=rep_s(g) @ state0, time=time, dt=0.005)

traj_lagrangian = lagrangian(state_trajectory)
g_traj_lagrangian = lagrangian(g_state_trajectory)

In [4]:
from ipywidgets import interactive, widgets
from IPython.display import display
from ipywidgets import HBox, VBox


# Initialize opacity values
alpha_surface = 0.3
alpha_fill = 0.5

# Create a grid for q and q_dot
grid_points = 50
q_values = np.linspace(-np.pi, np.pi, grid_points)
q_dot_values = np.linspace(-2*np.pi, 2*np.pi, grid_points)
Q, dQ = np.meshgrid(q_values, q_dot_values)

# Flatten the Q and Q_DOT arrays and stack them to create a 2D array of states
state_grid_flat = np.vstack((Q.flatten(), dQ.flatten()))
# Compute the Lagrangian for each state using a vectorized operation
L_values_flat = lagrangian(state_grid_flat, mass=1.0, damping=0.0)
# Reshape the flat Lagrangian values back to the original grid shape
L_values = L_values_flat.reshape(Q.shape)

# Create the 3D surface plot
lagrangian_surface_trace = go.Surface(
    z=L_values, x=Q, y=dQ, opacity=alpha_surface, name='Lagrangian Surface',
    # contours={
    # "z": {"show": True, "start": 0, "end": np.max(L_values),
    #   "size": (np.max(L_values) - np.min(L_values)) / 10, "color": "white"}
    # },
    showscale=False,  # Remove colorbar
    showlegend=True
)

color_state = '#50C878'
color_lagrangian = '#FF8C00'
line_width = 10
# Plot the trajectory and corresponding Lagrangian
state_traj_trace = go.Scatter3d(x=state_trajectory[0], y=state_trajectory[1], z=0*traj_lagrangian,
                                mode='lines', name='state[t]', line=dict(width=line_width, color=color_state))

g_state_traj_trace = go.Scatter3d(x=g_state_trajectory[0], y=g_state_trajectory[1], z=0*traj_lagrangian,
                                  mode='lines', name='g · state[t]', line=dict(width=line_width, dash='dash', color=color_state))

# Plot the trajectory and corresponding Lagrangian
lagrangian_traj_trace = go.Scatter3d(x=state_trajectory[0], y=state_trajectory[1], z=traj_lagrangian,
                                     mode='lines', name='L(q,dq)', line=dict(width=line_width, color=color_lagrangian))

g_lagrangian_traj_trace = go.Scatter3d(x=g_state_trajectory[0], y=g_state_trajectory[1], z=g_traj_lagrangian,
                                       mode='lines', name='L(g·q,g·dq)', line=dict(width=line_width, dash='dash', color=color_lagrangian))


# Create the surface for the "area under the curve"
# Interpolating between z=0 and the Lagrangian values
num_interpolations = 2
z_interpolations = np.linspace(0, 1, num_interpolations)[:, np.newaxis]
z_surface = z_interpolations * traj_lagrangian
g_z_surface = z_interpolations * g_traj_lagrangian

# Create the surface
auc_trace = go.Surface(x=np.tile(state_trajectory[0], (num_interpolations, 1)),
                       y=np.tile(state_trajectory[1], (num_interpolations, 1)),
                       z=z_surface,
                       opacity=alpha_fill,
                       showscale=False,
                       colorscale=[[0, '#90EE90'], [1, '#90EE90']],
                       name='traj_energy',
                       showlegend=True,
                       )
g_auc_trace = go.Surface(x=np.tile(g_state_trajectory[0], (num_interpolations, 1)),
                         y=np.tile(
                             g_state_trajectory[1], (num_interpolations, 1)),
                         z=g_z_surface,
                         opacity=alpha_fill*(0.5),
                         showscale=False,
                         colorscale=[[0, '#90EE90'], [1, '#90EE90']],
                         name='g·traj_energy',
                         showlegend=True,
                         )


# # Define the corner points for the plane
# q_min, q_max = np.min(q_values), np.max(q_values)
# dq_min, dq_max = np.min(q_dot_values), np.max(q_dot_values)
# L_min, L_max = np.min(L_values), np.max(L_values)

# lower_left_coord = [q_min, dq_max, 0]
# lower_right_coord = [q_max, dq_min, 0]
# upper_left_coord = [q_min, dq_max, L_max]
# upper_right_coord = [q_max, dq_min, L_max]
# plane_corners = np.array(
#     [lower_left_coord, lower_right_coord, upper_right_coord, upper_left_coord])

# # Define the vertex connectivities to form two triangles that make up the rectangular plane
# i = [0, 2]  # Vertex i (A, C)
# j = [1, 3]  # Vertex j (B, D)
# k = [2, 0]  # Vertex k (C, A)

# reflection_plane_trace = go.Mesh3d(
#     x=plane_corners[:, 0],
#     y=plane_corners[:, 1],
#     z=plane_corners[:, 2],
#     i=i,
#     j=j,
#     k=k,
#     opacity=0.2,
#     color='darkslategray',
#     name='Symmetry Plane',
#     showlegend=True,
# )

# Create initial plot
fig = go.FigureWidget(data=[lagrangian_surface_trace,
                      state_traj_trace, g_state_traj_trace, lagrangian_traj_trace, g_lagrangian_traj_trace, auc_trace, g_auc_trace])

fig.update_layout(scene=dict(
    xaxis_title=r'q',
    yaxis_title=r'dq',
    zaxis_title=r'L(q, dq)'),
    title='Lagrangian of a Simple Pendulum',
    height=800,
    showlegend=True)


def update_plot(max_time, epsilon, init_pos, init_vel, mass, damping, non_equiv_forcing):
    num_sinusoidal = 3
    time = max_time 
    state0 = np.asarray([init_pos, init_vel])
    print(f"Updating plot {state0}")
    # Recalculate data based on new parameters
    # Compute the Lagrangian for each state using a vectorized operation
    L_values_flat = lagrangian(
        state_grid_flat, mass=mass, damping=damping, non_equiv_offset=non_equiv_forcing)
    # Reshape the flat Lagrangian values back to the original grid shape
    L_values = L_values_flat.reshape(Q.shape)
    # Compute a new state trajectory
    state_trajectory = evolve_dynamics(
        state0=state0, time=time, dt=0.005, mass=mass, damping=damping, non_equiv_offset=non_equiv_forcing)
    # Compute the symmetric state trajectory
    g_state_trajectory = evolve_dynamics(
        state0=rep_s(g) @ state0, time=time, dt=0.005, mass=mass, damping=damping, non_equiv_offset=non_equiv_forcing)

    # traj_lagrangian = lagrangian(state_trajectory)
    var_q, var_dq = generate_trajectory_variation(state_trajectory,
                                                  epsilon=epsilon,
                                                  num_sinusoidal=num_sinusoidal)
    var_state_trajectory = state_trajectory + np.array([var_q, var_dq])
    var_g_state_trajectory = g_state_trajectory + np.array([var_q, var_dq])

    traj_lagrangian = lagrangian(
        var_state_trajectory, mass=mass, damping=damping, non_equiv_offset=non_equiv_forcing)
    g_traj_lagrangian = lagrangian(
        var_g_state_trajectory, mass=mass, damping=damping, non_equiv_offset=non_equiv_forcing)

    # Update lagrangian surface
    fig.data[0].update(z=L_values,
                       contours={
                           "z": {"show": True, "start": 0, "end": np.max(L_values),
                                 "size": (np.max(L_values) - np.min(L_values)) / 4, "color": "white"}
                       }
                       )
    # Update state trajectory
    fig.data[1].update(x=var_state_trajectory[0], y=var_state_trajectory[1])
    fig.data[2].update(x=var_g_state_trajectory[0],
                       y=var_g_state_trajectory[1])
    # Update lagrangian trajectory
    fig.data[3].update(x=var_state_trajectory[0],
                       y=var_state_trajectory[1], z=traj_lagrangian)
    fig.data[4].update(x=var_g_state_trajectory[0],
                       y=var_g_state_trajectory[1], z=g_traj_lagrangian)

    # Update area under curve
    # Interpolating between z=0 and the Lagrangian values
    num_interpolations = 10
    z_interpolations = np.linspace(0, 1, num_interpolations)[:, np.newaxis]
    z_surface = z_interpolations * traj_lagrangian
    g_z_surface = z_interpolations * g_traj_lagrangian
    # Create the surface
    fig.data[5].update(x=np.tile(var_state_trajectory[0], (num_interpolations, 1)),
                       y=np.tile(var_state_trajectory[1], (num_interpolations, 1)),
                       z=z_surface)
    fig.data[6].update(x=np.tile(var_g_state_trajectory[0], (num_interpolations, 1)),
                       y=np.tile(var_g_state_trajectory[1], (num_interpolations, 1)),
                       z=g_z_surface)


time_slider = widgets.FloatSlider(
    value=1.5,
    min=0.1,
    max=4,
    step=0.3,
    description='Time',
)

epsilon_slider = widgets.FloatSlider(
    value=0.0,
    min=0,
    max=0.01,
    step=0.001,
    description='Epsilon:',
)

non_equiv_slider = widgets.FloatSlider(
    value=0.0,
    min=0,
    max=6,
    step=1,
    description='Non-Equiv Forcing:',
)

init_pos_slider = widgets.FloatSlider(
    value=np.pi/2,
    min=-np.pi,
    max=np.pi,
    step=0.1,
    description='Initial Pos:',
)

init_vel_slider = widgets.FloatSlider(
    value=0,
    min=-2*np.pi,
    max=2*np.pi,
    step=0.1,
    description='Initial Vel:',
)

mass_slider = widgets.FloatSlider(
    value=1,
    min=0.1,
    max=10,
    step=0.1,
    description='Mass:',
)

damping_slider = widgets.FloatSlider(
    value=0.7,
    min=0,
    max=10.0,
    step=1,
    description='Damping:',
)

interactive_plot = interactive(
    update_plot,
    max_time=time_slider,
    epsilon=epsilon_slider,
    init_pos=init_pos_slider,
    init_vel=init_vel_slider,
    non_equiv_forcing=non_equiv_slider,
    mass=mass_slider,
    damping=damping_slider)

# Get the children of the interactive plot, which will include the sliders and the output
sliders = interactive_plot.children[:-1]

fig.update_layout(width=900)
# sliders.layout = Layout(width=.3)
# Place sliders and plot in custom layout
layout = HBox([VBox(sliders), fig])

# Display the layout
layout